In [2]:
# from __future__ import absolute_import, division, print_function

import pandas as pd
import numpy as np

# import tensorflow as tf
# tf.enable_eager_execution()

import os
import time
import re

In [2]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)  

In [3]:
all_drinks = pd.read_csv('all_drinks.csv')
all_drinks.head()

,Unnamed: 0,strDrink,dateModified,idDrink,strAlcoholic,strCategory,strDrinkThumb,strGlass,strIBA,strIngredient1,strIngredient10,strIngredient11,strIngredient12,strIngredient13,strIngredient14,strIngredient15,strIngredient2,strIngredient3,strIngredient4,strIngredient5,strIngredient6,strIngredient7,strIngredient8,strIngredient9,strInstructions,strMeasure1,strMeasure10,strMeasure11,strMeasure12,strMeasure13,strMeasure14,strMeasure15,strMeasure2,strMeasure3,strMeasure4,strMeasure5,strMeasure6,strMeasure7,strMeasure8,strMeasure9,strVideo
0,0,'57 Chevy with a White License Plate,2016-07-18 22:49:04,14029,Alcoholic,Cocktail,http://www.thecocktaildb.com/images/media/drink/qyyvtu1468878544.jpg,Highball glass,NaN,Creme de Cacao,NaN,NaN,NaN,NaN,NaN,NaN,Vodka,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1. Fill a rocks glass with ice 2.add white creme de cacao and vodka 3.stir,1 oz white,NaN,NaN,NaN,NaN,NaN,NaN,1 oz,,,,,,NaN,NaN,NaN
1,1,1-900-FUK-MEUP,2016-07-18 22:27:04,15395,Alcoholic,Shot,http://www.thecocktaildb.com/images/media/drink/uxywyw1468877224.jpg,Old-fashioned glass,NaN,Absolut Kurant,NaN,NaN,NaN,NaN,NaN,NaN,Grand Marnier,Chambord raspberry liqueur,Midori melon liqueur,Malibu rum,Amaretto,Cranberry juice,Pineapple juice,NaN,Shake ingredients in a mixing tin filled with ice cubes. Strain into a rocks glass.,1/2 oz,,,,,NaN,NaN,1/4 oz,1/4 oz,1/4 oz,1/4 oz,1/4 oz,1/2 oz,1/4 oz,,NaN
2,2,110 in the shade,2016-02-03 14:51:57,15423,Alcoholic,Beer,http://www.thecocktaildb.com/images/media/drink/xxyywq1454511117.jpg,Beer Glass,NaN,Lager,NaN,NaN,NaN,NaN,NaN,NaN,Tequila,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Drop shooter in glass. Fill with beer,16 oz,NaN,NaN,NaN,NaN,NaN,NaN,1.5 oz,,,,,,NaN,NaN,NaN
3,3,151 Florida Bushwacker,2016-07-18 22:28:43,14588,Alcoholic,Milk / Float / Shake,http://www.thecocktaildb.com/images/media/drink/rvwrvv1468877323.jpg,Beer mug,NaN,Malibu rum,NaN,NaN,NaN,NaN,NaN,NaN,Light rum,151 proof rum,Dark Creme de Cacao,Cointreau,Milk,Coconut liqueur,Vanilla ice-cream,NaN,Combine all ingredients. Blend until smooth. Garnish with chocolate shavings if desired.,1/2 oz,,,,,NaN,NaN,1/2 oz,1/2 oz Bacardi,1 oz,1 oz,3 oz,1 oz,1 cup,,NaN
4,4,155 Belmont,2016-10-05 12:36:28,15346,Alcoholic,Cocktail,http://www.thecocktaildb.com/images/media/drink/yqvvqs1475667388.jpg,White wine glass,NaN,Dark rum,NaN,NaN,NaN,NaN,NaN,NaN,Light rum,Vodka,Orange juice,NaN,NaN,NaN,NaN,NaN,Blend with ice. Serve in a wine glass. Garnish with carrot.,1 shot,NaN,NaN,NaN,NaN,NaN,NaN,2 shots,1 shot,1 shot,,,,,,NaN


In [42]:
ad_cols = list(all_drinks)
ad_cols[9:-1]

['strIngredient1',
 'strIngredient10',
 'strIngredient11',
 'strIngredient12',
 'strIngredient13',
 'strIngredient14',
 'strIngredient15',
 'strIngredient2',
 'strIngredient3',
 'strIngredient4',
 'strIngredient5',
 'strIngredient6',
 'strIngredient7',
 'strIngredient8',
 'strIngredient9',
 'strInstructions',
 'strMeasure1',
 'strMeasure10',
 'strMeasure11',
 'strMeasure12',
 'strMeasure13',
 'strMeasure14',
 'strMeasure15',
 'strMeasure2',
 'strMeasure3',
 'strMeasure4',
 'strMeasure5',
 'strMeasure6',
 'strMeasure7',
 'strMeasure8',
 'strMeasure9']

In [53]:
drinks = {
   'name': all_drinks['strDrink'],
    'instructions': all_drinks['strInstructions'],
    'ing1': all_drinks['strIngredient1'],
    'meas1': all_drinks['strMeasure1'],
    'ing2': all_drinks['strIngredient2'],
    'meas2': all_drinks['strMeasure2'],
    'ing3': all_drinks['strIngredient3'],
    'meas3': all_drinks['strMeasure3'],
    'ing4': all_drinks['strIngredient4'],
    'meas4': all_drinks['strMeasure4'],
    'ing5': all_drinks['strIngredient5'],
    'meas5': all_drinks['strMeasure5'],
    'ing6': all_drinks['strIngredient6'],
    'meas6': all_drinks['strMeasure6'],
    'ing7': all_drinks['strIngredient7'],
    'meas7': all_drinks['strMeasure7'],
    'ing8': all_drinks['strIngredient8'],
    'meas8': all_drinks['strMeasure8'],
    'ing9': all_drinks['strIngredient9'],
    'meas9': all_drinks['strMeasure9'],
    'ing10': all_drinks['strIngredient10'],
    'meas10': all_drinks['strMeasure10'],
    'ing11': all_drinks['strIngredient11'],
    'meas11': all_drinks['strMeasure11'],
    'ing12': all_drinks['strIngredient12'],
    'meas12': all_drinks['strMeasure12'],
    'ing13': all_drinks['strIngredient13'],
    'meas13': all_drinks['strMeasure13'],
    'ing14': all_drinks['strIngredient14'],
    'meas14': all_drinks['strMeasure14'],
    'ing15': all_drinks['strIngredient15'],
    'meas15': all_drinks['strMeasure15']
}
drink_df = pd.DataFrame(drinks)
drink_df

,name,instructions,ing1,meas1,ing2,meas2,ing3,meas3,ing4,meas4,ing5,meas5,ing6,meas6,ing7,meas7,ing8,meas8,ing9,meas9,ing10,meas10,ing11,meas11,ing12,meas12,ing13,meas13,ing14,meas14,ing15,meas15
0,'57 Chevy with a White License Plate,1. Fill a rocks glass with ice 2.add white creme de cacao and vodka 3.stir,Creme de Cacao,1 oz white,Vodka,1 oz,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1-900-FUK-MEUP,Shake ingredients in a mixing tin filled with ice cubes. Strain into a rocks glass.,Absolut Kurant,1/2 oz,Grand Marnier,1/4 oz,Chambord raspberry liqueur,1/4 oz,Midori melon liqueur,1/4 oz,Malibu rum,1/4 oz,Amaretto,1/4 oz,Cranberry juice,1/2 oz,Pineapple juice,1/4 oz,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,NaN,NaN,NaN
2,110 in the shade,Drop shooter in glass. Fill with beer,Lager,16 oz,Tequila,1.5 oz,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,151 Florida Bushwacker,Combine all ingredients. Blend until smooth. Garnish with chocolate shavings if desired.,Malibu rum,1/2 oz,Light rum,1/2 oz,151 proof rum,1/2 oz Bacardi,Dark Creme de Cacao,1 oz,Cointreau,1 oz,Milk,3 oz,Coconut liqueur,1 oz,Vanilla ice-cream,1 cup,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,NaN,NaN,NaN
4,155 Belmont,Blend with ice. Serve in a wine glass. Garnish with carrot.,Dark rum,1 shot,Light rum,2 shots,Vodka,1 shot,Orange juice,1 shot,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,24k nightmare,"Add over ice,shake and pour.",Goldschlager,1/2 oz,Jägermeister,1/2 oz,Rumple Minze,1/2 oz,151 proof rum,1/2 oz Bacardi,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,252,"Add both ingredients to shot glass, shoot, and get drunk quick",151 proof rum,1/2 shot Bacardi,Wild Turkey,1/2 shot,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,3 Wise Men,put them them in a glass... and slam it to tha head.,Jack Daniels,1/3 oz,Johnnie Walker,1/3 oz,Jim Beam,1/3 oz,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,3-Mile Long Island Iced Tea,Fill 14oz glass with ice and alcohol. Fill 2/3 glass with cola and remainder with sweet & sour. Top with dash of bitters and lemon wedge.,Gin,1/2 oz,Light rum,1/2 oz,Tequila,1/2 oz,Triple sec,1/2 oz,Vodka,1/2 oz,Coca-Cola,\n,Sweet and sour,1-2 dash,Bitters,1 wedge,Lemon,,NaN,,NaN,,NaN,,NaN,,NaN,NaN,NaN,NaN
9,410 Gone,NaN,Peach Vodka,2-3 oz,Coca-Cola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
d_cols = list(drink_df)
len(d_cols)

32

In [54]:
i1 = ''
    i1 = '' 
    i2 = ''
    i3 = ''
    i4 = ''
    i5 = ''
    i6 = ''
    i7 = ''
    i8 = ''
    i9 = ''
    i10 = ''
    i11 = ''
    i12 = ''
    i13 = ''
    i14 = ''
    i15 = ''

TypeError: 'int' object is not iterable

In [81]:
recipes2 = {}
for index, row in drink_df.iterrows():
    name = drink_df.loc[index, 'name']
    recipes2.setdefault(name, {})
    recipes2[name]['ingredients'] = []
    recipes2[name]['instructions'] = drink_df.loc[index, 'instructions']

    if pd.notnull(drink_df.loc[index, d_cols[2]]):
        m1 = drink_df.loc[index, d_cols[2]]
    else:
        m1 = ''
    if pd.notnull(drink_df.loc[index, d_cols[3]]):
        i1 = drink_df.loc[index, d_cols[3]]
    else:
        i1 = ''
    if pd.notnull(drink_df.loc[index, d_cols[4]]):
        m2 = drink_df.loc[index, d_cols[4]]
    else:
        m2 = ''
    if pd.notnull(drink_df.loc[index, d_cols[5]]):
        i2 = drink_df.loc[index, d_cols[5]]
    else:
        i2 = ''
    if pd.notnull(drink_df.loc[index, d_cols[6]]):
        m3 = drink_df.loc[index, d_cols[6]]
    else:
        m3 = ''
    if pd.notnull(drink_df.loc[index, d_cols[7]]):
        i3 = drink_df.loc[index, d_cols[7]]
    else:
        i3 = ''
    if pd.notnull(drink_df.loc[index, d_cols[8]]):
        m4 = drink_df.loc[index, d_cols[8]]
    else:
        m4 = ''
    if pd.notnull(drink_df.loc[index, d_cols[9]]):
        i4 = drink_df.loc[index, d_cols[9]]
    else:
        i4 = ''
    if pd.notnull(drink_df.loc[index, d_cols[10]]):
        m5 = drink_df.loc[index, d_cols[10]]
    else:
        m5 = ''
    if pd.notnull(drink_df.loc[index, d_cols[11]]):
        i5 = drink_df.loc[index, d_cols[11]]
    else:
        i5 = ''
    if pd.notnull(drink_df.loc[index, d_cols[12]]):
        m6 = drink_df.loc[index, d_cols[12]]
    else:
        m6 = ''
    if pd.notnull(drink_df.loc[index, d_cols[13]]):
        i6 = drink_df.loc[index, d_cols[13]]
    else:
        i6 = ''
    if pd.notnull(drink_df.loc[index, d_cols[14]]):
        m7 = drink_df.loc[index, d_cols[14]]
    else:
        m7 = ''
    if pd.notnull(drink_df.loc[index, d_cols[15]]):
        i7 = drink_df.loc[index, d_cols[15]]
    else:
        i7 = ''
    if pd.notnull(drink_df.loc[index, d_cols[16]]):
        m8 = drink_df.loc[index, d_cols[16]]
    else:
        m8 = ''
    if pd.notnull(drink_df.loc[index, d_cols[17]]):
        i8 = drink_df.loc[index, d_cols[17]]
    else:
        i8 = ''
    if pd.notnull(drink_df.loc[index, d_cols[18]]):
        m9 = drink_df.loc[index, d_cols[18]]
    else:
        m9 = ''
    if pd.notnull(drink_df.loc[index, d_cols[19]]):
        i9 = drink_df.loc[index, d_cols[19]]
    else:
        i9 = ''
    if pd.notnull(drink_df.loc[index, d_cols[20]]):
        m10 = drink_df.loc[index, d_cols[20]]
    else:
        m10 = ''
    if pd.notnull(drink_df.loc[index, d_cols[21]]):
        i10 = drink_df.loc[index, d_cols[21]]
    else:
        i10 = ''
    if pd.notnull(drink_df.loc[index, d_cols[22]]):
        m11 = drink_df.loc[index, d_cols[22]]
    else:
        m11 = ''
    if pd.notnull(drink_df.loc[index, d_cols[23]]):
        i11 = drink_df.loc[index, d_cols[23]]
    else:
        i11 = ''
    if pd.notnull(drink_df.loc[index, d_cols[24]]):
        m12 = drink_df.loc[index, d_cols[24]]
    else:
        m12 = ''
    if pd.notnull(drink_df.loc[index, d_cols[25]]):
        i12 = drink_df.loc[index, d_cols[25]]
    else:
        i12 = ''
    if pd.notnull(drink_df.loc[index, d_cols[26]]):
        m13 = drink_df.loc[index, d_cols[26]]
    else:
        m13 = ''
    if pd.notnull(drink_df.loc[index, d_cols[27]]):
        i13 = drink_df.loc[index, d_cols[27]]
    else:
        i3 = ''
    if pd.notnull(drink_df.loc[index, d_cols[28]]):
        m14 = drink_df.loc[index, d_cols[28]]
    else:
        m14= ''
    if pd.notnull(drink_df.loc[index, d_cols[29]]):
        i14 = drink_df.loc[index, d_cols[29]]
    else:
        i14 = ''
    if pd.notnull(drink_df.loc[index, d_cols[30]]):
        m15 = drink_df.loc[index, d_cols[30]]
    else:
        m15 = ''
    if pd.notnull(drink_df.loc[index, d_cols[31]]):
        i15 = drink_df.loc[index, d_cols[31]]
    else:
        i15 = ''

        
    if (m1 and i1) != ('' or ' '):
        recipes2[name]['ingredients'].append(str(m1) + ' ' + str(i1))
    elif i1 != ('' or ' '):
        recipes2[name]['ingredients'].append(str(i1))
    if (m2 and i2) != ('' or ' '):
        recipes2[name]['ingredients'].append(str(m2) + ' ' + str(i2))
    elif i2 != ('' or ' '):
        recipes2[name]['ingredients'].append(str(i2))
    if (m3 and i3) != ('' or ' '):
        recipes2[name]['ingredients'].append(str(m3) + ' ' + str(i3))
    elif i3 != ('' or ' '):
        recipes2[name]['ingredients'].append(str(i3))
    if (m4 and i4) != ('' or ' '):
        recipes2[name]['ingredients'].append(str(m4) + ' ' + str(i4))
    elif i4 != ('' or ' '):
        recipes2[name]['ingredients'].append(str(i4))
    if (m5 and i5) != ('' or ' '):
        recipes2[name]['ingredients'].append(str(m5) + ' ' + str(i5))
    elif i5 != ('' or ' '):
        recipes2[name]['ingredients'].append(str(i5))
    if (m6 and i6) != ('' or ' '):
        recipes2[name]['ingredients'].append(str(m6) + ' ' + str(i6))
    elif i6 != ('' or ' '):
        recipes2[name]['ingredients'].append(str(i6))
    if (m7 and i7) != ('' or ' '):
        recipes2[name]['ingredients'].append(str(m7) + ' ' + str(i7))
    elif i7 != ('' or ' '):
        recipes2[name]['ingredients'].append(str(i7))
    if (m8 and i8) != ('' or ' '):
        recipes2[name]['ingredients'].append(str(m8) + ' ' + str(i8))
    elif i8 != ('' or ' '):
        recipes2[name]['ingredients'].append(str(i8))
    if (m9 and i9) != ('' or ' '):
        recipes2[name]['ingredients'].append(str(m9) + ' ' + str(i9))
    elif i9 != ('' or ' '):
        recipes2[name]['ingredients'].append(str(i9))
    if (m10 and i10) != ('' or ' '):
        recipes2[name]['ingredients'].append(str(m10) + ' ' + str(i10))
    if (m11 and i11) != ('' or ' '):
        recipes2[name]['ingredients'].append(str(m11) + ' ' + str(i11))
    elif i11 != ('' or ' '):
        recipes2[name]['ingredients'].append(str(i11))
    if (m12 and i12) != ('' or ' '):
        recipes2[name]['ingredients'].append(str(m12) + ' ' + str(i12))
    elif i12 != ('' or ' '):
        recipes2[name]['ingredients'].append(str(i12))
    if (m13 and i13) != ('' or ' '):
        recipes2[name]['ingredients'].append(str(m13) + ' ' + str(i13))
    elif i13 != ('' or ' '):
        recipes2[name]['ingredients'].append(str(i13))
    if (m14 and i14) != ('' or ' '):
        recipes2[name]['ingredients'].append(str(m14) + ' ' + str(i14))
    elif i14 != ('' or ' '):
        recipes2[name]['ingredients'].append(str(i14))
    if (m15 and i15) != ('' or ' '):
        recipes2[name]['ingredients'].append(str(m15) + ' ' + str(i15))
    elif i15 != ('' or ' '):
        recipes2[name]['ingredients'].append(str(i15))
        
    

In [ ]:
# recipes2 = {}
# r = re.compile('\s+')
# for index, row in drink_df.iterrows():
#     name = drink_df.loc[index, 'name']
#     recipes2.setdefault(name, {})
#     recipes2[name]['ingredients'] = []
#     recipes2[name]['instructions'] = drink_df.loc[index, 'instructions']
#     i1 = ''
#     i1 = '' 
#     i2 = ''
#     i3 = ''
#     i4 = ''
#     i5 = ''
#     i6 = ''
#     i7 = ''
#     i8 = ''
#     i9 = ''
#     i10 = ''
#     i11 = ''
#     i12 = ''
#     i13 = ''
#     i14 = ''
#     i15 = ''
#     m1 = ''
#     m1 = '' 
#     m2 = ''
#     m3 = ''
#     m4 = ''
#     m5 = ''
#     m6 = ''
#     m7 = ''
#     m8 = ''
#     m9 = ''
#     m10 = ''
#     m11 = ''
#     m12 = ''
#     m13 = ''
#     m14 = ''
#     m15 = ''
#     if pd.notnull(drink_df.loc[index, d_cols[2]]):
#         m1 = drink_df.loc[index, d_cols[2]]
#     if pd.notnull(drink_df.loc[index, d_cols[3]]):
#         i1 = drink_df.loc[index, d_cols[3]]
#     if pd.notnull(drink_df.loc[index, d_cols[4]]):
#         m2 = drink_df.loc[index, d_cols[4]]
#     if pd.notnull(drink_df.loc[index, d_cols[5]]):
#         i2 = drink_df.loc[index, d_cols[5]]
#     if pd.notnull(drink_df.loc[index, d_cols[6]]):
#         m3 = drink_df.loc[index, d_cols[6]]
#     if pd.notnull(drink_df.loc[index, d_cols[7]]):
#         i3 = drink_df.loc[index, d_cols[7]]
#     if pd.notnull(drink_df.loc[index, d_cols[8]]):
#         m4 = drink_df.loc[index, d_cols[8]]
#     if pd.notnull(drink_df.loc[index, d_cols[9]]):
#         i4 = drink_df.loc[index, d_cols[9]]
#     if pd.notnull(drink_df.loc[index, d_cols[10]]):
#         m5 = drink_df.loc[index, d_cols[10]]
#     if pd.notnull(drink_df.loc[index, d_cols[11]]):
#         i5 = drink_df.loc[index, d_cols[11]]
#     if pd.notnull(drink_df.loc[index, d_cols[12]]):
#         m6 = drink_df.loc[index, d_cols[12]]
#     if pd.notnull(drink_df.loc[index, d_cols[13]]):
#         i6 = drink_df.loc[index, d_cols[13]]
#     if pd.notnull(drink_df.loc[index, d_cols[14]]):
#         m7 = drink_df.loc[index, d_cols[14]]
#     if pd.notnull(drink_df.loc[index, d_cols[15]]):
#         i7 = drink_df.loc[index, d_cols[15]]
#     if pd.notnull(drink_df.loc[index, d_cols[16]]):
#         m8 = drink_df.loc[index, d_cols[16]]
#     if pd.notnull(drink_df.loc[index, d_cols[17]]):
#         i8 = drink_df.loc[index, d_cols[17]]
#     if pd.notnull(drink_df.loc[index, d_cols[18]]):
#         m9 = drink_df.loc[index, d_cols[18]]
#     if pd.notnull(drink_df.loc[index, d_cols[19]]):
#         i9 = drink_df.loc[index, d_cols[19]]
#     if pd.notnull(drink_df.loc[index, d_cols[20]]):
#         m10 = drink_df.loc[index, d_cols[20]]
#     if pd.notnull(drink_df.loc[index, d_cols[21]]):
#         i10 = drink_df.loc[index, d_cols[21]]
#     if pd.notnull(drink_df.loc[index, d_cols[22]]):
#         m11 = drink_df.loc[index, d_cols[22]]
#     if pd.notnull(drink_df.loc[index, d_cols[23]]):
#         i11 = drink_df.loc[index, d_cols[23]]
#     if pd.notnull(drink_df.loc[index, d_cols[24]]):
#         m12 = drink_df.loc[index, d_cols[24]]
#     if pd.notnull(drink_df.loc[index, d_cols[25]]):
#         i12 = drink_df.loc[index, d_cols[25]]
#     if pd.notnull(drink_df.loc[index, d_cols[26]]):
#         m13 = drink_df.loc[index, d_cols[26]]
#     if pd.notnull(drink_df.loc[index, d_cols[27]]):
#         i13 = drink_df.loc[index, d_cols[27]]
#     if pd.notnull(drink_df.loc[index, d_cols[28]]):
#         m14 = drink_df.loc[index, d_cols[28]]
#     if pd.notnull(drink_df.loc[index, d_cols[29]]):
#         i14 = drink_df.loc[index, d_cols[29]]
#     if pd.notnull(drink_df.loc[index, d_cols[30]]):
#         m15 = drink_df.loc[index, d_cols[30]]
#     if pd.notnull(drink_df.loc[index, d_cols[31]]):
#         i15 = drink_df.loc[index, d_cols[31]]

        
#     if (m1 and i1) != ('' or '  '):
#         recipes2[name]['ingredients'].append(str(m1) + '  ' + str(i1))
#     elif i1 != ('' or '  '):
#         recipes2[name]['ingredients'].append(str(i1))
#     if (m2 and i2) != ('' or '  '):
#         recipes2[name]['ingredients'].append(str(m2) + '  ' + str(i2))
#     elif i2 != ('' or '  '):
#         recipes2[name]['ingredients'].append(str(i2))
#     if (m3 and i3) != ('' or '  '):
#         recipes2[name]['ingredients'].append(str(m3) + '  ' + str(i3))
#     elif i3 != ('' or '  '):
#         recipes2[name]['ingredients'].append(str(i3))
#     if (m4 and i4) != ('' or '  '):
#         recipes2[name]['ingredients'].append(str(m4) + '  ' + str(i4))
#     elif i4 != ('' or '  '):
#         recipes2[name]['ingredients'].append(str(i4))
#     if (m5 and i5) != ('' or '  '):
#         recipes2[name]['ingredients'].append(str(m5) + '  ' + str(i5))
#     elif i5 != ('' or '  '):
#         recipes2[name]['ingredients'].append(str(i5))
#     if (m6 and i6) != ('' or '  '):
#         recipes2[name]['ingredients'].append(str(m6) + '  ' + str(i6))
#     elif i6 != ('' or '  '):
#         recipes2[name]['ingredients'].append(str(i6))
#     if (m7 and i7) != ('' or '  '):
#         recipes2[name]['ingredients'].append(str(m7) + '  ' + str(i7))
#     elif i7 != ('' or '  '):
#         recipes2[name]['ingredients'].append(str(i7))
#     if (m8 and i8) != ('' or '  '):
#         recipes2[name]['ingredients'].append(str(m8) + '  ' + str(i8))
#     elif i8 != ('' or '  '):
#         recipes2[name]['ingredients'].append(str(i8))
#     if (m9 and i9) != ('' or '  '):
#         recipes2[name]['ingredients'].append(str(m9) + '  ' + str(i9))
#     elif i9 != ('' or '  '):
#         recipes2[name]['ingredients'].append(str(i9))
#     if (m10 and i10) != ('' or '  '):
#         recipes2[name]['ingredients'].append(str(m10) + '  ' + str(i10))
#     if (m11 and i11) != ('' or '  '):
#         recipes2[name]['ingredients'].append(str(m11) + '  ' + str(i11))
#     elif i11 != ('' or '  '):
#         recipes2[name]['ingredients'].append(str(i11))
#     if (m12 and i12) != ('' or '  '):
#         recipes2[name]['ingredients'].append(str(m12) + '  ' + str(i12))
#     elif i12 != ('' or '  '):
#         recipes2[name]['ingredients'].append(str(i12))
#     if (m13 and i13) != ('' or '  '):
#         recipes2[name]['ingredients'].append(str(m13) + '  ' + str(i13))
#     elif i13 != ('' or '  '):
#         recipes2[name]['ingredients'].append(str(i13))
#     if (m14 and i14) != ('' or '  '):
#         recipes2[name]['ingredients'].append(str(m14) + '  ' + str(i14))
#     elif i14 != ('' or '  '):
#         recipes2[name]['ingredients'].append(str(i14))
#     if (m15 and i15) != ('' or '  '):
#         recipes2[name]['ingredients'].append(str(m15) + '  ' + str(i15))
#     elif i15 != ('' or '  '):
#         recipes2[name]['ingredients'].append(str(i15))
        
    

In [36]:
# drink_df = pd.DataFrame({col: all_drinks[col] for col in ad_cols[9:-1]})
# drink_df['name'] = all_drinks[ad_cols[1]]
# drink_df.head()

,strIngredient1,strIngredient10,strIngredient11,strIngredient12,strIngredient13,strIngredient14,strIngredient15,strIngredient2,strIngredient3,strIngredient4,strIngredient5,strIngredient6,strIngredient7,strIngredient8,strIngredient9,strInstructions,strMeasure1,strMeasure10,strMeasure11,strMeasure12,strMeasure13,strMeasure14,strMeasure15,strMeasure2,strMeasure3,strMeasure4,strMeasure5,strMeasure6,strMeasure7,strMeasure8,strMeasure9,name
0,Creme de Cacao,NaN,NaN,NaN,NaN,NaN,NaN,Vodka,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1. Fill a rocks glass with ice 2.add white creme de cacao and vodka 3.stir,1 oz white,NaN,NaN,NaN,NaN,NaN,NaN,1 oz,,,,,,NaN,NaN,'57 Chevy with a White License Plate
1,Absolut Kurant,NaN,NaN,NaN,NaN,NaN,NaN,Grand Marnier,Chambord raspberry liqueur,Midori melon liqueur,Malibu rum,Amaretto,Cranberry juice,Pineapple juice,NaN,Shake ingredients in a mixing tin filled with ice cubes. Strain into a rocks glass.,1/2 oz,,,,,NaN,NaN,1/4 oz,1/4 oz,1/4 oz,1/4 oz,1/4 oz,1/2 oz,1/4 oz,,1-900-FUK-MEUP
2,Lager,NaN,NaN,NaN,NaN,NaN,NaN,Tequila,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Drop shooter in glass. Fill with beer,16 oz,NaN,NaN,NaN,NaN,NaN,NaN,1.5 oz,,,,,,NaN,NaN,110 in the shade
3,Malibu rum,NaN,NaN,NaN,NaN,NaN,NaN,Light rum,151 proof rum,Dark Creme de Cacao,Cointreau,Milk,Coconut liqueur,Vanilla ice-cream,NaN,Combine all ingredients. Blend until smooth. Garnish with chocolate shavings if desired.,1/2 oz,,,,,NaN,NaN,1/2 oz,1/2 oz Bacardi,1 oz,1 oz,3 oz,1 oz,1 cup,,151 Florida Bushwacker
4,Dark rum,NaN,NaN,NaN,NaN,NaN,NaN,Light rum,Vodka,Orange juice,NaN,NaN,NaN,NaN,NaN,Blend with ice. Serve in a wine glass. Garnish with carrot.,1 shot,NaN,NaN,NaN,NaN,NaN,NaN,2 shots,1 shot,1 shot,,,,,,155 Belmont


In [82]:
recipes2

{"'57 Chevy with a White License Plate": {'ingredients': ['Creme de Cacao 1 oz white ',
   'Vodka 1 oz ',
   ' ',
   '  ',
   '  ',
   '  ',
   '  ',
   ' ',
   ' ',
   ' ',
   ' ',
   ' ',
   '  ',
   ' ',
   ' '],
  'instructions': '1. Fill a rocks glass with ice 2.add white creme de cacao and vodka 3.stir'},
 '1-900-FUK-MEUP': {'ingredients': ['Absolut Kurant 1/2 oz ',
   'Grand Marnier 1/4 oz ',
   'Chambord raspberry liqueur 1/4 oz ',
   'Midori melon liqueur 1/4 oz ',
   'Malibu rum 1/4 oz ',
   'Amaretto 1/4 oz ',
   'Cranberry juice 1/2 oz ',
   'Pineapple juice 1/4 oz ',
   '  ',
   '  ',
   '  ',
   '  ',
   '  ',
   ' ',
   ' '],
  'instructions': 'Shake ingredients in a mixing tin filled with ice cubes. Strain into a rocks glass.'},
 '110 in the shade': {'ingredients': ['Lager 16 oz ',
   'Tequila 1.5 oz ',
   ' ',
   '  ',
   '  ',
   '  ',
   '  ',
   ' ',
   ' ',
   ' ',
   ' ',
   ' ',
   '  ',
   ' ',
   ' '],
  'instructions': 'Drop shooter in glass. Fill with beer'},

In [3]:
boston_drinks = pd.read_csv('mr-boston-flattened.csv')#, engine='python', encoding='utf-8')
boston_drinks.head()


,name,category,measurement-1,ingredient-1,measurement-2,ingredient-2,measurement-3,ingredient-3,measurement-4,ingredient-4,measurement-5,ingredient-5,measurement-6,ingredient-6,instructions,glass,glass-size
0,Gauguin,Cocktail Classics,2 oz,Light Rum,1 oz,Passion Fruit Syrup,1 oz,Lemon Juice,1 oz,Lime Juice,NaN,NaN,NaN,NaN,Combine ingredients with a cup of crushed ice ...,Old-Fashioned Glass,6 to 8 ounces
1,Fort Lauderdale,Cocktail Classics,1 1/2 oz,Light Rum,1/2 oz,Sweet Vermouth,1/4 oz,Juice of Orange,1/4 oz,Juice of a Lime,NaN,NaN,NaN,NaN,Shake with ice and strain into old-fashioned g...,Old-Fashioned Glass,6 to 8 ounces
2,Apple Pie,Cordials and Liqueurs,3 oz,Apple schnapps,1 oz,Cinnamon schnapps,NaN,Apple slice,NaN,NaN,NaN,NaN,NaN,NaN,Pour into ice-filled old-fashioned glass. Garn...,Old-Fashioned Glass,6 to 8 ounces
3,Cuban Cocktail No. 1,Cocktail Classics,1/2 oz,Juice of a Lime,1/2 oz,Powdered Sugar,2 oz,Light Rum,NaN,NaN,NaN,NaN,NaN,NaN,Shake with ice and strain into cocktail glass.,Cocktail Glass,6 or more ounces
4,Cool Carlos,Cocktail Classics,1 1/2 oz,Dark rum,2 oz,Cranberry Juice,2 oz,Pineapple Juice,1 oz,Orange curacao,1 oz,Sour Mix,NaN,NaN,"Mix all ingredients except curacao with ice, s...",Collins Glass,14 to 16 ounces


In [10]:
boston_drinks.loc[2, cols[6]]

nan

In [11]:
pd.notnull(boston_drinks.loc[2, cols[6]])

False

In [12]:
if pd.isnull(boston_drinks.loc[2, cols[6]]):
    print('test')

test


In [7]:
cols = list(boston_drinks)
cols

['name',
 'category',
 'measurement-1',
 'ingredient-1',
 'measurement-2',
 'ingredient-2',
 'measurement-3',
 'ingredient-3',
 'measurement-4',
 'ingredient-4',
 'measurement-5',
 'ingredient-5',
 'measurement-6',
 'ingredient-6',
 'instructions',
 'glass',
 'glass-size']

In [8]:
recipes = {}
for index, row in boston_drinks.iterrows():
    name = boston_drinks.loc[index, 'name']
    recipes.setdefault(name, {})
    recipes[name]['ingredients'] = []
    recipes[name]['instructions'] = boston_drinks.loc[index, cols[14]]
    if pd.notnull(boston_drinks.loc[index, cols[2]]):
        m1 = boston_drinks.loc[index, cols[2]]
    if pd.notnull(boston_drinks.loc[index, cols[3]]):
        i1 = boston_drinks.loc[index, cols[3]]
    if pd.notnull(boston_drinks.loc[index, cols[4]]):
        m2 = boston_drinks.loc[index, cols[4]]
    if pd.notnull(boston_drinks.loc[index, cols[5]]):
        i2 = boston_drinks.loc[index, cols[5]]
    if pd.notnull(boston_drinks.loc[index, cols[6]]):
        m3 = boston_drinks.loc[index, cols[6]]
    if pd.notnull(boston_drinks.loc[index, cols[7]]):
        i3 = boston_drinks.loc[index, cols[7]]
    if pd.notnull(boston_drinks.loc[index, cols[8]]):
        m4 = boston_drinks.loc[index, cols[8]]
    if pd.notnull(boston_drinks.loc[index, cols[9]]):
        i4 = boston_drinks.loc[index, cols[9]]
    if pd.notnull(boston_drinks.loc[index, cols[10]]):
        m5 = boston_drinks.loc[index, cols[10]]
    if pd.notnull(boston_drinks.loc[index, cols[11]]):
        i5 = boston_drinks.loc[index, cols[11]]
    if pd.notnull(boston_drinks.loc[index, cols[12]]):
        m6 = boston_drinks.loc[index, cols[12]]
    if pd.notnull(boston_drinks.loc[index, cols[13]]):
        i6 = boston_drinks.loc[index, cols[13]]
    if m1 and i1:
        recipes[name]['ingredients'].append(str(m1) + ' ' + str(i1))
    elif i1:
        recipes[name]['ingredients'].append(str(i1))
    if m2 and i2:
        recipes[name]['ingredients'].append(str(m2) + ' ' + str(i2))
    elif i2:
        recipes[name]['ingredients'].append(str(i2))
    if m3 and i3:
        recipes[name]['ingredients'].append(str(m3) + ' ' + str(i3))
    elif i3:
        recipes[name]['ingredients'].append(str(i3))
    if m4 and i4:
        recipes[name]['ingredients'].append(str(m4) + ' ' + str(i4))
    elif i4:
        recipes[name]['ingredients'].append(str(i4))
    if m5 and i5:
        recipes[name]['ingredients'].append(str(m5) + ' ' + str(i5))
    elif i5:
        recipes[name]['ingredients'].append(str(i5))
    if m6 and i6:
        recipes[name]['ingredients'].append(str(m6) + ' ' + str(i6))
    elif i6:
        recipes[name]['ingredients'].append(str(i6))
        
    

NameError: name 'm5' is not defined

In [4]:
boston_drinks.fillna('')

,name,category,measurement-1,ingredient-1,measurement-2,ingredient-2,measurement-3,ingredient-3,measurement-4,ingredient-4,measurement-5,ingredient-5,measurement-6,ingredient-6,instructions,glass,glass-size
0,Gauguin,Cocktail Classics,2 oz,Light Rum,1 oz,Passion Fruit Syrup,1 oz,Lemon Juice,1 oz,Lime Juice,,,,,Combine ingredients with a cup of crushed ice ...,Old-Fashioned Glass,6 to 8 ounces
1,Fort Lauderdale,Cocktail Classics,1 1/2 oz,Light Rum,1/2 oz,Sweet Vermouth,1/4 oz,Juice of Orange,1/4 oz,Juice of a Lime,,,,,Shake with ice and strain into old-fashioned g...,Old-Fashioned Glass,6 to 8 ounces
2,Apple Pie,Cordials and Liqueurs,3 oz,Apple schnapps,1 oz,Cinnamon schnapps,,Apple slice,,,,,,,Pour into ice-filled old-fashioned glass. Garn...,Old-Fashioned Glass,6 to 8 ounces
3,Cuban Cocktail No. 1,Cocktail Classics,1/2 oz,Juice of a Lime,1/2 oz,Powdered Sugar,2 oz,Light Rum,,,,,,,Shake with ice and strain into cocktail glass.,Cocktail Glass,6 or more ounces
4,Cool Carlos,Cocktail Classics,1 1/2 oz,Dark rum,2 oz,Cranberry Juice,2 oz,Pineapple Juice,1 oz,Orange curacao,1 oz,Sour Mix,,,"Mix all ingredients except curacao with ice, s...",Collins Glass,14 to 16 ounces
5,John Collins,Whiskies,2 oz,Bourbon whiskey,1 oz,Fresh lemon juice,1/2 oz,Simple Syrup,1 oz,Soda water,1 oz,"Orange and lemon wheels, maraschino cherry",,,Shake first three ingredients with ice and str...,Collins Glass,14 to 16 ounces
6,Cherry Rum,Cocktail Classics,1 1/4 oz,Light Rum,1 1/2 oz,cherry-flavored brandy,1 oz,Light cream,,,,,,,Shake with ice and strain into cocktail glass.,Cocktail Glass,6 or more ounces
7,Casa Blanca,Cocktail Classics,2 oz,Light Rum,1 1/2 oz,Lime Juice,1 1/2 oz,Triple Sec,1 1/2 oz,Maraschino,,,,,Shake with ice and strain into cocktail glass.,Cocktail Glass,6 or more ounces
8,Caribbean Champagne,Cocktail Classics,1/2 oz,Light Rum,1/2 oz,Creme de banana,,Chilled Champagne,,,,,,,Pour rum and banana liqueur into champagne flu...,Champagne Flute,6 to 8 ounces
9,Amber Amour,Cordials and Liqueurs,1 1/2 oz,Amaretto,1/4 oz,Fresh lemon juice,1/4 oz,Simple Syrup,,Soda water,Maraschino cherry,,,,"Pour amaretto, lemon juice, and syrup into ice...",Collins Glass,14 to 16 ounces


In [9]:
recipes = {}
for index, row in boston_drinks.iterrows():
    name = boston_drinks.loc[index, 'name']
    recipes.setdefault(name, {})
    recipes[name]['ingredients'] = []
    recipes[name]['instructions'] = boston_drinks.loc[index, cols[14]]
    if (boston_drinks.loc[index, cols[2]] != '') and (boston_drinks.loc[index, cols[3]] != ''):
        recipes[name]['ingredients'].append(str(boston_drinks.loc[index, cols[2]]) + ' ' + str(boston_drinks.loc[index, cols[3]]))
    elif (boston_drinks.loc[index, cols[3]] != ''):
        recipes[name]['ingredients'].append(str(boston_drinks.loc[index, cols[3]]))
    if (boston_drinks.loc[index, cols[4]] != '') and (boston_drinks.loc[index, cols[5]] != ''):
        recipes[name]['ingredients'].append(str(boston_drinks.loc[index, cols[4]]) + ' ' + str(boston_drinks.loc[index, cols[5]]))
    elif (boston_drinks.loc[index, cols[5]] != ''):
        recipes[name]['ingredients'].append(str(boston_drinks.loc[index, cols[3]]))
    if (boston_drinks.loc[index, cols[6]] != '') and (boston_drinks.loc[index, cols[7]] != ''):
        recipes[name]['ingredients'].append(str(boston_drinks.loc[index, cols[6]]) + ' ' + str(boston_drinks.loc[index, cols[7]]))
    elif (boston_drinks.loc[index, cols[7]] != ''):
        recipes[name]['ingredients'].append(str(boston_drinks.loc[index, cols[7]]))
    if (boston_drinks.loc[index, cols[8]] != '') and (boston_drinks.loc[index, cols[9]] != ''):
        recipes[name]['ingredients'].append(str(boston_drinks.loc[index, cols[8]]) + ' ' + str(boston_drinks.loc[index, cols[9]]))
    elif (boston_drinks.loc[index, cols[9]] != ''):
        recipes[name]['ingredients'].append(str(boston_drinks.loc[index, cols[9]]))
    if (boston_drinks.loc[index, cols[10]] != '') and (boston_drinks.loc[index, cols[11]] != ''):
        recipes[name]['ingredients'].append(str(boston_drinks.loc[index, cols[10]]) + ' ' + str(boston_drinks.loc[index, cols[11]]))
    elif (boston_drinks.loc[index, cols[11]] != ''):
        recipes[name]['ingredients'].append(str(boston_drinks.loc[index, cols[11]]))
    if (boston_drinks.loc[index, cols[12]] != '') and (boston_drinks.loc[index, cols[13]] != ''):
        recipes[name]['ingredients'].append(str(boston_drinks.loc[index, cols[12]]) + ' ' + str(boston_drinks.loc[index, cols[13]]))
    elif (boston_drinks.loc[index, cols[13]] != ''):
        recipes[name]['ingredients'].append(str(boston_drinks.loc[index, cols[13]]))


In [10]:
len(recipes)

990